In [1]:
import tensorflow as tf
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
class_number_to_str_name = [
    "person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck",
    "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench",
    "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe",
    "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard",
    "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard",
    "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl",
    "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza",
    "donut", "cake", "chair", "sofa", "potted plant", "bed", "dining table", "toilet",
    "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone", "microwave",
    "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
    "teddy bear", "hair drier", "toothbrush"
]

In [13]:
default_model = "./yolo_v4_tiny_float/default_yolo_v4_tiny_float.tflite"
samsung_model = "./yolo_v4_tiny_float/samsung_yolo_v4_tiny_float.tflite"

model_path = default_model

interpreter = tf.lite.Interpreter(model_path=model_path)

interpreter.allocate_tensors()

In [16]:
# 모델 분석

# 입력 및 출력 정보 얻기
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# 입력 텐서 정보 출력
print("== input tensor information ==")
for i, detail in enumerate(input_details):
    print(f"input tensor {i}:")
    print(f"\tname: {detail['name']}")
    print(f"\tshape: {detail['shape']}")
    print(f"\tdtype: {detail['dtype']}")
    print()

# 출력 텐서 정보 출력
print("== output tensor information ==")
for i, detail in enumerate(output_details):
    print(f"output tensor {i}:")
    print(f"\tname: {detail['name']}")
    print(f"\tshape: {detail['shape']}")
    print(f"\tdtype: {detail['dtype']}")
    print()

== input tensor information ==
input tensor 0:
	name: input_1
	shape: [  1 416 416   3]
	dtype: <class 'numpy.float32'>

== output tensor information ==
output tensor 0:
	name: Identity
	shape: [   1 2535    4]
	dtype: <class 'numpy.float32'>

output tensor 1:
	name: Identity_1
	shape: [   1 2535   80]
	dtype: <class 'numpy.float32'>



In [5]:
# 입력 데이터 (입력 이미지) 전처리
image_path = "./images/4.jpg"

input_size = input_details[0]['shape'][1:3] # model shape 정보에서 size를 추출
input_width = input_details[0]['shape'][1]
input_height = input_details[0]['shape'][2]

input_data = Image.open(image_path)
input_data = input_data.resize(([input_width, input_height]))
input_data = np.array(input_data, dtype=np.float32)
input_data = input_data / 255.0  # YOLO는 일반적으로 [0, 1] 범위로 정규화
input_data = np.expand_dims(input_data, axis=0)  # 배치 차원 추가

In [6]:
# 입력 텐서 설정
interpreter.set_tensor(input_details[0]['index'], input_data)

# 추론 실행
interpreter.invoke()

In [7]:
# 출력 텐서 가져오기
output_boxes = interpreter.get_tensor(output_details[0]['index'])[0]  # [1, 2535, 4] -> [2535, 4]
output_classes = interpreter.get_tensor(output_details[1]['index'])[0]  # [1, 2535, 80] -> [2535, 80]

print(output_boxes)

# 원본 이미지 로드 및 변환
original_image = cv2.imread(image_path)
original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

original_width = original_image.shape[0]
original_height = original_image.shape[1]

[[ 11.205271    8.568879   25.071056   18.65446  ]
 [ 23.220482    6.3585234  49.515987   13.703494 ]
 [ 39.930122    5.454539   73.388664   11.937492 ]
 ...
 [328.1985    386.62402   219.00336   195.39273  ]
 [362.05118   386.25354   213.56212   212.73016  ]
 [397.54865   388.2361    230.18303   257.90527  ]]


In [8]:
# YOLOv4 Tiny 모델 후처리
def postprocess_boxes(boxes, classes, confidence_threshold=0.2):
    # 박스와 클래스 정보에서 유의미한 정보만 추출
    valid_boxes = []
    for i in range(len(boxes)):
        if np.max(classes[i] > confidence_threshold):  # confidence score
            valid_boxes.append({
                'box': boxes[i][:4],  # x, y, width, height
                'probability': max(classes[i]),
                'class': np.argmax(classes[i]),  # 가장 높은 클래스 확률  
                'name': class_number_to_str_name[np.argmax(classes[i])]
            })
    return valid_boxes

# 후처리된 박스 출력
detected_boxes = postprocess_boxes(output_boxes, output_classes)

for box in detected_boxes:
    x_min = min(box['box'][0], box['box'][2])
    x_max = max(box['box'][0], box['box'][2])
    y_min = min(box['box'][1], box['box'][3])
    y_max = max(box['box'][1], box['box'][3])
    box['box'][0] = x_min
    box['box'][2] = x_max
    box['box'][1] = y_min
    box['box'][3] = y_max
    print(box)

{'box': array([131.46312, 110.70185, 251.89836, 156.60028], dtype=float32), 'probability': 0.7988769, 'class': 49, 'name': 'orange'}
{'box': array([135.63538, 113.65448, 258.25247, 160.58606], dtype=float32), 'probability': 0.3193094, 'class': 49, 'name': 'orange'}
{'box': array([ 90.86511, 167.35991, 122.65942, 204.02307], dtype=float32), 'probability': 0.3541014, 'class': 49, 'name': 'orange'}
{'box': array([100.974434, 165.16595 , 141.24101 , 224.87541 ], dtype=float32), 'probability': 0.4926079, 'class': 49, 'name': 'orange'}
{'box': array([142.25644, 204.9604 , 201.738  , 213.08353], dtype=float32), 'probability': 0.4677621, 'class': 49, 'name': 'orange'}
{'box': array([132.04755, 199.53432, 190.04976, 231.3411 ], dtype=float32), 'probability': 0.25123793, 'class': 49, 'name': 'orange'}
{'box': array([134.86867, 197.0836 , 197.43951, 229.68745], dtype=float32), 'probability': 0.63491607, 'class': 49, 'name': 'orange'}
{'box': array([240.25185, 184.23856, 286.31326, 346.6111 ], dty

In [9]:
# (선택 사항) 결과 이미지 시각화
def draw_boxes(image, boxes, x_ratio, y_ratio, class_names):
    for box in boxes:
        # x_min, y_min, x_max, y_max = map(int, box['box'])
        x_min, y_min, width, height = box['box']
        
        x_min = int(x_min * x_ratio)
        y_min = int(y_min * y_ratio)
        width = int(width * x_ratio)
        height = int(height * y_ratio)
        
        class_id = box['class']
        # confidence = box['score']
        color = (0, 255, 0)  # 녹색
        # label = f"Class {class_id} ({confidence:.2f})"
        cv2.rectangle(image, (x_min, y_min), (width, height), color, 2)
        # cv2.putText(image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
    return image


# print(original_width, original_height, input_width, input_height)
# 이미지에 박스 그리기

image_with_boxes = draw_boxes(original_image_rgb, detected_boxes, (original_width / input_width), (original_height / input_height), class_names=None)
cv2.imshow('Detected Objects', image_with_boxes)
cv2.waitKey(0)
cv2.destroyAllWindows()